In [ ]:
#importing libraries
import os
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Image preprocessing [1]

In [ ]:
BATCH_SIZE = 64

train_generator = ImageDataGenerator(rotation_range=90, 
                                     brightness_range=[0.1, 0.7],
                                     width_shift_range=0.5, 
                                     height_shift_range=0.5,
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     validation_split=0.15,
                                     preprocessing_function=preprocess_input) 
#I have used 15 percent of data for validation

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) 

In [ ]:
#Loading images from directories
train_data_dir = " " #example file path: /content/drive/MyDrive/TrainingImages
test_data_dir = "  "

#Creating directory iterators for training, validation and testing
traingen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(224, 224),
                                               class_mode='categorical',                                               
                                               subset='training',
                                               batch_size=BATCH_SIZE, 
                                               shuffle=True,
                                               seed=42)

validgen = train_generator.flow_from_directory(train_data_dir,
                                               target_size=(224, 224),
                                               class_mode='categorical',                                              
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testgen = test_generator.flow_from_directory(test_data_dir,
                                             target_size=(224, 224),
                                             class_mode=None,                                             
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

Model Creation [2]

In [ ]:
def cnn_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):

    # conv_base = InceptionV3(
    #     include_top=False,
    #     weights="imagenet",
    #     input_shape=input_shape
    #     )

    conv_base = Xception(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape
        )

#     conv_base = VGG16(
#         include_top=False,
#         weights="imagenet",
#         input_shape=input_shape
#         )

#code to switch the layers between trainable and non-trainable

    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

#Creating a new fully connected top model 

    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
# Grouping the convolutional base and new fully-connected layers into a Model object.

    model = Model(inputs=conv_base.input, outputs=output_layer)

# Compiling the model for training

    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

Model training with no fine tuning (excercing a pretrained model behavior)

In [ ]:
input_shape = (224, 224, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes=16

n_steps = traingen.samples // BATCH_SIZE
n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 15

model = cnn_model(input_shape, n_classes, optim_1, fine_tune=0)

Setting up liveloss plot [3]

In [ ]:
pip install livelossplot

In [ ]:
from livelossplot.inputs.keras import PlotLossesCallback

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

In [ ]:
%%time 
plot_loss_1 = PlotLossesCallback()

history = model.fit(traingen,
                            batch_size=BATCH_SIZE,
                            epochs=n_epochs,
                            validation_data=validgen,
                            steps_per_epoch=n_steps,
                            validation_steps=n_val_steps,
                            callbacks=[plot_loss_1, early_stop],
                            )

In [ ]:
#Generating predictions [4]
true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())

test_predict = model.predict(testgen)
test_pred_classes = np.argmax(test_predict, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
#Printing the accuracy
test_acc = accuracy_score(true_classes, test_pred_classes)
print("Model Accuracy without Fine-Tuning: {:.2f}%".format(test_acc * 100))

Model training with fine tuning

In [ ]:
# Resetting our image data generators
traingen.reset()
validgen.reset()
testgen.reset()

# Using a smaller learning rate
optim_2 = Adam(lr=0.0001)

# Re-compiling the model leaving the last 2 layers unfrozen for Fine-Tuning
model_ft = cnn_model(input_shape, n_classes, optim_2, fine_tune=2)

In [ ]:
# #this part is left for testing
# input_shape = (224, 224, 3)
# optim_1 = Adam(learning_rate=0.001)
# n_classes=16

# n_steps = traingen.samples // BATCH_SIZE
# n_val_steps = validgen.samples // BATCH_SIZE
# n_epochs = 15

# optim_3 = Adam(lr=0.00001)

# model_ft3 = cnn_model(input_shape, n_classes, optim_3, fine_tune=2)

In [ ]:
%%time

plot_loss_2 = PlotLossesCallback()

# Retrain model with fine-tuning
history_ft = model_ft.fit(traingen,
                                  batch_size=BATCH_SIZE,
                                  epochs=n_epochs,
                                  validation_data=validgen,
                                  steps_per_epoch=n_steps, 
                                  validation_steps=n_val_steps,
                                  callbacks=[plot_loss_2, early_stop],
                                  verbose=1)

In [ ]:
#Generating model predictions
test_predict_ft = model_ft.predict(testgen)
pred_classes_ft = np.argmax(test_predict_ft, axis=1)

In [ ]:
#printing the accuracy results
from sklearn.metrics import accuracy_score

true_classes = testgen.classes
class_indices = traingen.class_indices
class_indices = dict((v,k) for k,v in class_indices.items())


model_acc_ft = accuracy_score(true_classes,pred_classes_ft)
print("Model Accuracy with Fine-Tuning: {:.2f}%".format(model_acc_ft * 100))

In [ ]:
#Predicting and Saving it as a CSV file
import pandas as pd
y_pred = model_ft.predict(testgen)
y_pred = np.argmax(y_pred, axis=1)
y_pred = pd.Series(y_pred, name='Class')
sub = pd.concat([pd.Series(range(1, 482), name="Id"), y_pred], axis=1)
sub.to_csv("/content/drive/MyDrive/CollabFiles/test2.csv", index=False)

**References:**

[1]https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
[1]https://www.analyticsvidhya.com/blog/2021/01/building-a-cnn-model-with-95-accuracy/
[2]https://keras.io/api/applications/#finetune-inceptionv3-on-a-new-set-of-classes
[3]https://pypi.org/project/livelossplot/
[4]https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/